In [1]:
# importa as bibliotecas necessárias
import PyPDF2
import re
import pandas as pd

In [2]:
# Abre o arquivo pdf 
path = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Contratos\FI_Contrato_Romilton_Assinatura Digital-Manifesto.pdf'
pdf_file = open(path, 'rb')


In [3]:
#Faz a leitura usando a biblioteca
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,6):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)

In [4]:
text

"           INSTRUMENTO PARTICULAR  DE COMPRA E VENDA DE IMÓVEL, FINANCIAMENTO COM PACTO  ADJETO  DE  ALIENAÇÃO FIDUCIÁRIA  EM  GARANTIA, EMISSÃO  DE CÉDULA DE CRÉDITO IMOBILIÁRIO  – CCI E OUTRAS AVENÇAS  Nº 0001133420/RC   Esta página é parte integrante e inseparável do Instrumento Particular de Compra e Venda de Imóvel, Financiamento com Pacto Adjeto de Alienação Fiduciária em Garantia, Emissão de Cédula de Crédito Imobiliário – CCI e Outras Avenças nº 0001133420/RC   CAMPO 1 – CREDORA RAZÃO SOCIAL: QI SOCIEDADE DE CRÉDITO DIRETO S.A.    ENDEREÇO COMERCIAL: Avenida Brigadeiro Faria Lima, nº 2.391, 1º andar  COMPLEMENTO: Conjunto 12 BAIRRO: Jardim Paulistano CIDADE: São Paulo UF: SP   CEP: 01452-000  CNPJ: 32.402.502/0001-35  DATA DA CONSTITUIÇÃO: 10/01/2019  REGISTRO JUCESP: 35300530225   REPRESENTANTES: Roberto Amaral Dantas Junior, brasileiro, portador da cédula de identidade RG nº 50.016.943 -3 SSP/SP e inscrito no CPF sob nº 400.516.238 -03, nos termos da Procuração Pública lavra

In [34]:
listaDePara = {'valorTotal':'1. Valor do Financiamento: R$','tabela': 'Sistema de Amortização:','Taxa ao Mes':'2.1.1. Juros de','registro': '4.3. Despesas Cartorárias de Registro: R$','valorLiquido': '7. Valor Líquido a Liberar do  Financiamento : R$',
                'prazoMes': 'PRAZO DE AMORTIZAÇÃO :','valorPrimeiraParcelaComEncargos':'VALOR TOTAL DO PRIMEIRO ENCARGO, NESTA DATA:  R$',
                'valorImóvel':'Imóvel para fins de leilão:  R$','prazoContrato': 'N.º DE PRESTAÇÕES: ','ultimaParcela':'DATA DE VENCIMENTO DA ÚLTIMA PRESTAÇÃO: ','dataContrato': 'Data de Liberação dos Recursos: ',
                'valorPrimeiraParcela':'VALOR  DA  PARCELA  MENSAL  (AMORTIZAÇÃO  E  JUROS),  NESTA  DATA: R$','primeiraParcela':'DATA DE VENCIMENTO DA PRIMEIRA PRESTAÇÃO: ','indice':'ÍNDICE: MENSAL do'
                }

#len(listaHE) # <--- Qtd de Itens na Lista
listaKey = []
listaValues = []

for key, value in listaDePara.items():
    inicioFrase = text.find(value,0)
    finalFrase = inicioFrase + len(value) + 1
    proximoEspaco = text.find(" ", finalFrase)
    valorExtraido = text[finalFrase:proximoEspaco]

    #Ajustar Valores Númericos
    if '.' in valorExtraido:
        valorExtraido = valorExtraido.replace(".", "")
        valorExtraido = valorExtraido.replace(",", ".")
    
    #Ajustar Valores Percentuais
    if '%' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", ".")
        valorExtraido = valorExtraido.replace("%", "")
        valorExtraido = round(float(valorExtraido)/100,4)
    elif 'IPCA' or 'IGPM' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", "")
    
    elif key == '4.3. Despesas Cartorárias de Registro: R$':
        #Definir Variáveis Auxiliares
        topico4 = '4. Despesas'
        topico5 = '5. Valor Destinado'

        #Pegar posição das variáveis auxiliares no texto
        inicioTopico = text.find(topico4, 0)
        finalTopico = text.find(topico5, 0)

        #Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
        paragrafo4 = text[inicioTopico+len(topico4)+1:finalTopico-1]
        paragrafo4 = re.sub('\s+',' ', paragrafo4)
        paragrafo4

        listaChave = ['4.3.','4.4.']
        inicioItens = []

        for item in listaChave:
            inicioP = paragrafo4.find(item,0)
            inicioItens.append(inicioP)
            
        item1 = paragrafo4[inicioItens[0]:inicioItens[1]-1]

        listaFinal = [item1]
        print(listaFinal)
        listaValor = []

        for itemAux in listaFinal:
            if '[X]' in itemAux:
                inicioAux = itemAux.find('R$ ', 0)
                fimAux = itemAux.find(",", inicioAux) + 3
                valorExtraido = itemAux[inicioAux+3:fimAux]
            else:
                valorExtraido = '0,00'
    listaKey.append(key)
    listaValues.append(valorExtraido)


In [35]:
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue

{'valorTotal': '299726.00',
 'tabela': 'SAC',
 'Taxa ao Mes': 0.0068,
 'registro': '8276.00',
 'valorLiquido': '270000.00',
 'prazoMes': '360',
 'valorPrimeiraParcelaComEncargos': '3022.80',
 'valorImóvel': '491000.00',
 'prazoContrato': '359',
 'ultimaParcela': '15/07/2052',
 'dataContrato': '05/07/2022',
 'valorPrimeiraParcela': '2900.07',
 'primeiraParcela': '15/09/2022',
 'indice': 'IPCA'}